In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, LogisticRegression, Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, classification_report, roc_auc_score
import joblib
import pandas as pd
import numpy as np

线性回归实战--预测房价

In [2]:
# 加载数据集
hp = load_boston()

C:\Users\life\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the C

In [3]:
print('获取特征值:', hp.data)
print('获取目标值:', hp.target)

获取特征值: [[6.3200e-03 1.8000e+01 2.3100e+00 ... 1.5300e+01 3.9690e+02 4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9690e+02 9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9283e+02 4.0300e+00]
 ...
 [6.0760e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 5.6400e+00]
 [1.0959e-01 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9345e+02 6.4800e+00]
 [4.7410e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 7.8800e+00]]
获取目标值: [24.  21.6 34.7 33.4 36.2 28.7 22.9 27.1 16.5 18.9 15.  18.9 21.7 20.4
 18.2 19.9 23.1 17.5 20.2 18.2 13.6 19.6 15.2 14.5 15.6 13.9 16.6 14.8
 18.4 21.  12.7 14.5 13.2 13.1 13.5 18.9 20.  21.  24.7 30.8 34.9 26.6
 25.3 24.7 21.2 19.3 20.  16.6 14.4 19.4 19.7 20.5 25.  23.4 18.9 35.4
 24.7 31.6 23.3 19.6 18.7 16.  22.2 25.  33.  23.5 19.4 22.  17.4 20.9
 24.2 21.7 22.8 23.4 24.1 21.4 20.  20.8 21.2 20.3 28.  23.9 24.8 22.9
 23.9 26.6 22.5 22.2 23.6 28.7 22.6 22.  22.9 25.  20.6 28.4 21.4 38.7
 43.8 33.2 27.5 26.5 18.6 19.3 20.1 19.5 19.5

In [4]:
print('获取特征名:', hp.feature_names)

获取特征名: ['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


In [5]:
print(hp.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [6]:
# 分割数据集
x_train, x_test, y_train, y_test = train_test_split(hp.data, hp.target, test_size=0.3, random_state=1)
print(x_train)

[[6.29760e-01 0.00000e+00 8.14000e+00 ... 2.10000e+01 3.96900e+02
  8.26000e+00]
 [1.71710e-01 2.50000e+01 5.13000e+00 ... 1.97000e+01 3.78080e+02
  1.44400e+01]
 [9.82349e+00 0.00000e+00 1.81000e+01 ... 2.02000e+01 3.96900e+02
  2.12400e+01]
 ...
 [5.87205e+00 0.00000e+00 1.81000e+01 ... 2.02000e+01 3.96900e+02
  1.93700e+01]
 [3.30450e-01 0.00000e+00 6.20000e+00 ... 1.74000e+01 3.76750e+02
  1.08800e+01]
 [8.01400e-02 0.00000e+00 5.96000e+00 ... 1.92000e+01 3.96900e+02
  8.77000e+00]]


In [7]:
# 特征值不在同一个量纲内，所以标准化处理
std = StandardScaler()
x_train = std.fit_transform(x_train)
x_test = std.transform(x_test)

In [8]:
# 分类不存在目标值标准化，回归会对目标值进行了标准化
std_y = StandardScaler()
#变维reshape()：-1代表把剩余的元素都堆到哪一维
temp = y_train.reshape(-1, 1)
# 标签只有一个一个房价，是一维的
y_train = std_y.fit_transform(y_train.reshape(-1, 1))  # 目标值是一维的，这里需要传进去2维的
print(y_train.shape)
y_test = std_y.transform(y_test.reshape(-1, 1))
print(y_test.shape)

(354, 1)
(152, 1)


预测

In [9]:
lr = LinearRegression()
# 训练
lr.fit(x_train, y_train)
print('回归系数:', lr.coef_) # 看特征与目标之间的相关性
# 预测
y_predict = lr.predict(x_test)
# 获取真实房价,逆标准化
y_lr_predict = std_y.inverse_transform(y_predict)

回归系数: [[-0.09309616  0.15852628  0.04498381  0.07540369 -0.28082703  0.21461817
   0.01119884 -0.35915378  0.30000376 -0.21278503 -0.23925274  0.06462201
  -0.45883502]]


array([[32.3914647 ],
       [27.94401279],
       [17.83762764],
       [21.66941388],
       [18.93639614],
       [20.01995795],
       [32.63414961],
       [17.86179976],
       [24.73211676],
       [26.77346826],
       [27.26138023],
       [28.63243014],
       [21.16103038],
       [27.15098589],
       [23.33198614],
       [21.02894005],
       [17.11819755],
       [37.52455938],
       [30.69864427],
       [ 8.46104486],
       [20.83966032],
       [16.26732451],
       [25.05030307],
       [24.8069032 ],
       [31.6789299 ],
       [10.98989363],
       [13.54111562],
       [16.54697099],
       [35.86699683],
       [14.69589266],
       [21.05655107],
       [13.90301353],
       [43.01321262],
       [17.80335305],
       [21.68817023],
       [20.45077825],
       [17.38095983],
       [26.81334904],
       [10.08706217],
       [19.96174586],
       [24.1983249 ],
       [21.00202622],
       [29.38682967],
       [16.33215467],
       [19.37439646],
       [14

In [10]:
# 保存训练好的模型，模型中保存的是w的值，也保存了模型结构
joblib.dump(lr, './test.pkl')
#加载模型
# model = joblib_load('./test.pkl')

['./test.pkl']

In [12]:
# 使用正规方程预测价格
print('使用正规方程预测的房价:', y_lr_predict)
# 回归只有测试集误差:误差越小越好
print('均方误差:', mean_squared_error(y_test,y_predict))

使用正规方程预测的房价: [[32.3914647 ]
 [27.94401279]
 [17.83762764]
 [21.66941388]
 [18.93639614]
 [20.01995795]
 [32.63414961]
 [17.86179976]
 [24.73211676]
 [26.77346826]
 [27.26138023]
 [28.63243014]
 [21.16103038]
 [27.15098589]
 [23.33198614]
 [21.02894005]
 [17.11819755]
 [37.52455938]
 [30.69864427]
 [ 8.46104486]
 [20.83966032]
 [16.26732451]
 [25.05030307]
 [24.8069032 ]
 [31.6789299 ]
 [10.98989363]
 [13.54111562]
 [16.54697099]
 [35.86699683]
 [14.69589266]
 [21.05655107]
 [13.90301353]
 [43.01321262]
 [17.80335305]
 [21.68817023]
 [20.45077825]
 [17.38095983]
 [26.81334904]
 [10.08706217]
 [19.96174586]
 [24.1983249 ]
 [21.00202622]
 [29.38682967]
 [16.33215467]
 [19.37439646]
 [14.45525235]
 [39.52670798]
 [18.18535021]
 [26.38536303]
 [20.41420863]
 [25.08511581]
 [24.40208165]
 [24.87902769]
 [26.97887956]
 [ 4.93921657]
 [24.0001365 ]
 [10.66711602]
 [26.98494775]
 [16.63507993]
 [35.31652989]
 [19.49249915]
 [27.37577863]
 [16.53965715]
 [19.09746715]
 [11.06765336]
 [32.0344526

array([[ 1.11554709],
       [ 0.62196148],
       [-0.49966197],
       [-0.07440395],
       [-0.37771881],
       [-0.25746332],
       [ 1.14248067],
       [-0.49697931],
       [ 0.26549991],
       [ 0.4920525 ],
       [ 0.54620178],
       [ 0.69836318],
       [-0.1308252 ],
       [ 0.53395003],
       [ 0.11011108],
       [-0.14548481],
       [-0.57950552],
       [ 1.68522649],
       [ 0.92767507],
       [-1.54029073],
       [-0.16649138],
       [-0.67393682],
       [ 0.30081276],
       [ 0.27379983],
       [ 1.0364688 ],
       [-1.25963489],
       [-0.97649602],
       [-0.64290119],
       [ 1.50126744],
       [-0.84833695],
       [-0.14242049],
       [-0.93633199],
       [ 2.29436636],
       [-0.50346582],
       [-0.07232234],
       [-0.20965016],
       [-0.55034372],
       [ 0.49647853],
       [-1.35983263],
       [-0.26392379],
       [ 0.2062588 ],
       [-0.14847175],
       [ 0.7820877 ],
       [-0.66674187],
       [-0.32910881],
       [-0

梯度下降

In [13]:
sgd = SGDRegressor()
sgd.fit(x_train,y_train)

C:\Users\life\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor()

In [15]:
# # 预测
y_pre = sgd.predict(x_test)
y_pre

array([ 0.92806123,  0.6594421 , -0.49447639,  0.0162673 , -0.43767716,
       -0.21124596,  0.94974451, -0.45683875,  0.18198768,  0.51579874,
        0.45461026,  0.79213888, -0.08216332,  0.44244309,  0.06826731,
       -0.28536025, -0.59001062,  1.7037842 ,  0.890271  , -1.42162729,
       -0.17089576, -0.58356046,  0.31658131,  0.31164545,  0.96736426,
       -1.31784755, -0.93659333, -0.40148532,  1.49660094, -0.92596016,
        0.11517186, -0.89259098,  2.11050993, -0.48994416,  0.16764309,
       -0.14434394, -0.55027192,  0.60784388, -1.52075334, -0.32498474,
        0.43894686, -0.06616041,  0.70964965, -0.77080013, -0.4108851 ,
       -0.85084862,  1.94865855, -0.50854582,  0.38872689, -0.2067734 ,
        0.2711905 ,  0.23197066,  0.34655066,  0.46536586, -1.72767091,
        0.16417198, -1.31114876,  0.48802037, -0.57283585,  1.47224088,
       -0.33524594,  0.57184493, -0.71475483, -0.4872227 , -1.24151394,
        1.02883863,  1.57226098,  0.26962662,  0.25150769,  0.30

In [17]:
# 回归系数
sgd.coef_

array([-0.06717674,  0.09813954, -0.01640762,  0.0842953 , -0.1913172 ,
        0.25484279, -0.01456697, -0.27251037,  0.12062218, -0.04735092,
       -0.22810544,  0.07393504, -0.43408871])

In [18]:
# 均方误差
mean_squared_error(y_test,y_pre)



0.24020736179881222